In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [2]:
#Untuk Google Colab :
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
file = '/content/drive/MyDrive/GoogleColab/news.csv'
dataset = pd.read_csv(file)

Mounted at /content/drive


In [3]:
dataset.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
dataset.tail()

,Unnamed: 0,title,text,label
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL
6334,4330,Jeb Bush Is Suddenly Attacking Trump. Here's W...,Jeb Bush Is Suddenly Attacking Trump. Here's W...,REAL


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [6]:
dataset.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
label,0


In [7]:
def clean_text(text):
    import re
    return re.sub(r'[^\w\s]', '', text.lower())

dataset['cleaned_text'] = dataset['text'].apply(clean_text)

def clean_text(text):
    import re
    return re.sub(r'[^\w\s]', '', text.lower())

dataset['cleaned_text'] = dataset['title'].apply(clean_text)

In [8]:
dataset.columns

Index(['Unnamed: 0', 'title', 'text', 'label', 'cleaned_text'], dtype='object')

In [9]:
dataset.loc[0, 'title']

'You Can Smell Hillary’s Fear'

In [10]:
dataset.loc[0, 'text']

'Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam. \nIn the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI. \nThe word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI. \nBut that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election. \nThe FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI. \nThe FBI’s leadership is being warned that the entir

In [11]:
dataset = dataset[['title', 'text']]
dataset.head()

,title,text
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello..."
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T..."
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...


In [12]:
dataset.columns = ['title', 'text']
dataset.head()

,title,text
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello..."
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T..."
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...


In [13]:
dataset.describe()

,title,text
count,6335,6335
unique,6256,6060
top,OnPolitics | 's politics blog,"Killing Obama administration rules, dismantlin..."
freq,5,58


In [14]:
dataset['text_len'] = dataset['text'].apply(len)
dataset.head(10)

,title,text,text_len
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",7518
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,2646
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,2543
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",2660
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1840
5,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",13333
6,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",3171
7,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,783
8,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,13863
9,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,4296


In [15]:
import re
import nltk
# Defining a function to clean up the text
def clean(Text):
    sms = re.sub('[^a-zA-Z]', ' ', Text) #Replacing all non-alphabetic characters with a space
    sms = sms.lower() #converting to lowecase
    sms = sms.split()
    sms = ' '.join(sms)
    return sms

dataset["Clean_Text"] = dataset["text"].apply(clean)
#Lets have a look at a sample of texts after cleaning
print("The First 10 Texts after cleaning: ",*dataset["Clean_Text"][:10], sep = "\n")

The First 10 Texts after cleaning: 
daniel greenfield a shillman journalism fellow at the freedom center is a new york writer focusing on radical islam in the final stretch of the election hillary rodham clinton has gone to war with the fbi the word unprecedented has been thrown around so often this election that it ought to be retired but it s still unprecedented for the nominee of a major political party to go war with the fbi but that s exactly what hillary and her people have done coma patients just waking up now and watching an hour of cnn from their hospital beds would assume that fbi director james comey is hillary s opponent in this election the fbi is under attack by everyone from obama to cnn hillary s people have circulated a letter attacking comey there are currently more media hit pieces lambasting him than targeting trump it wouldn t be too surprising if the clintons or their allies were to start running attack ads against the fbi the fbi s leadership is being warned that

In [16]:
dataset

,title,text,text_len,Clean_Text
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",7518,daniel greenfield a shillman journalism fellow...
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,2646,google pinterest digg linkedin reddit stumbleu...
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,2543,u s secretary of state john f kerry said monda...
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",2660,kaydee king kaydeeking november the lesson fro...
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1840,it s primary day in new york and front runners...
...,...,...,...,...
6330,State Department says it can't find emails fro...,The State Department told the Republican Natio...,4076,the state department told the republican natio...
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,14323,the p in pbs should stand for plutocratic or p...
6332,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,11974,anti trump protesters are tools of the oligarc...
6333,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",6991,addis ababa ethiopia president obama convened ...


In [17]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
import os
os.environ['NLTK_DATA'] = '/usr/local/nltk_data'
nltk.download('punkt', download_dir='/usr/local/nltk_data')

[nltk_data] Downloading package punkt to /usr/local/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
import nltk
print(nltk.data.path)

['/root/nltk_data', '/usr/nltk_data', '/usr/share/nltk_data', '/usr/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


In [20]:
import nltk
import os

# Atur path untuk data NLTK
nltk.data.path.append('/usr/local/nltk_data')
nltk.download('punkt', download_dir='/usr/local/nltk_data')

[nltk_data] Downloading package punkt to /usr/local/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [22]:
dataset["Tokenize_Text"] = dataset.apply(lambda row: nltk.word_tokenize(row["Clean_Text"]), axis=1)
dataset.head(10)

,title,text,text_len,Clean_Text,Tokenize_Text
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",7518,daniel greenfield a shillman journalism fellow...,"[daniel, greenfield, a, shillman, journalism, ..."
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,2646,google pinterest digg linkedin reddit stumbleu...,"[google, pinterest, digg, linkedin, reddit, st..."
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,2543,u s secretary of state john f kerry said monda...,"[u, s, secretary, of, state, john, f, kerry, s..."
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",2660,kaydee king kaydeeking november the lesson fro...,"[kaydee, king, kaydeeking, november, the, less..."
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1840,it s primary day in new york and front runners...,"[it, s, primary, day, in, new, york, and, fron..."
5,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",13333,i m not an immigrant but my grandparents are m...,"[i, m, not, an, immigrant, but, my, grandparen..."
6,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",3171,share this baylee luciani left screenshot of w...,"[share, this, baylee, luciani, left, screensho..."
7,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,783,a czech stockbroker who saved more than jewish...,"[a, czech, stockbroker, who, saved, more, than..."
8,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,13863,hillary clinton and donald trump made some ina...,"[hillary, clinton, and, donald, trump, made, s..."
9,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,4296,iranian negotiators reportedly have made a las...,"[iranian, negotiators, reportedly, have, made,..."


In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [25]:
# Removing the stopwords function
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    filtered_text = [word for word in text if word not in stop_words]
    return filtered_text

dataset["Nostopword_Text"] = dataset["Tokenize_Text"].apply(remove_stopwords)

# print("The First 10 Texts after removing the stopwords:",*spam["Nostopword_Text"][:10], sep = "\n")

In [26]:
dataset

,title,text,text_len,Clean_Text,Tokenize_Text,Nostopword_Text
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",7518,daniel greenfield a shillman journalism fellow...,"[daniel, greenfield, a, shillman, journalism, ...","[daniel, greenfield, shillman, journalism, fel..."
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,2646,google pinterest digg linkedin reddit stumbleu...,"[google, pinterest, digg, linkedin, reddit, st...","[google, pinterest, digg, linkedin, reddit, st..."
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,2543,u s secretary of state john f kerry said monda...,"[u, s, secretary, of, state, john, f, kerry, s...","[u, secretary, state, john, f, kerry, said, mo..."
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",2660,kaydee king kaydeeking november the lesson fro...,"[kaydee, king, kaydeeking, november, the, less...","[kaydee, king, kaydeeking, november, lesson, t..."
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1840,it s primary day in new york and front runners...,"[it, s, primary, day, in, new, york, and, fron...","[primary, day, new, york, front, runners, hill..."
...,...,...,...,...,...,...
6330,State Department says it can't find emails fro...,The State Department told the Republican Natio...,4076,the state department told the republican natio...,"[the, state, department, told, the, republican...","[state, department, told, republican, national..."
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,14323,the p in pbs should stand for plutocratic or p...,"[the, p, in, pbs, should, stand, for, plutocra...","[p, pbs, stand, plutocratic, pentagon, posted,..."
6332,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,11974,anti trump protesters are tools of the oligarc...,"[anti, trump, protesters, are, tools, of, the,...","[anti, trump, protesters, tools, oligarchy, re..."
6333,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",6991,addis ababa ethiopia president obama convened ...,"[addis, ababa, ethiopia, president, obama, con...","[addis, ababa, ethiopia, president, obama, con..."


In [27]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [28]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [29]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
# lemmatize string
def lemmatize_word(text):
    #word_tokens = word_tokenize(text)
    # provide context i.e. part-of-speech
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in text]
    return lemmas

dataset["Lemmatized_Text"] = dataset["Nostopword_Text"].apply(lemmatize_word)
# print("The First 10 Texts after lemmatization:",*spam["Lemmatized_Text"][:10], sep = "\n")

In [30]:
dataset

,title,text,text_len,Clean_Text,Tokenize_Text,Nostopword_Text,Lemmatized_Text
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",7518,daniel greenfield a shillman journalism fellow...,"[daniel, greenfield, a, shillman, journalism, ...","[daniel, greenfield, shillman, journalism, fel...","[daniel, greenfield, shillman, journalism, fel..."
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,2646,google pinterest digg linkedin reddit stumbleu...,"[google, pinterest, digg, linkedin, reddit, st...","[google, pinterest, digg, linkedin, reddit, st...","[google, pinterest, digg, linkedin, reddit, st..."
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,2543,u s secretary of state john f kerry said monda...,"[u, s, secretary, of, state, john, f, kerry, s...","[u, secretary, state, john, f, kerry, said, mo...","[u, secretary, state, john, f, kerry, say, mon..."
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",2660,kaydee king kaydeeking november the lesson fro...,"[kaydee, king, kaydeeking, november, the, less...","[kaydee, king, kaydeeking, november, lesson, t...","[kaydee, king, kaydeeking, november, lesson, t..."
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,1840,it s primary day in new york and front runners...,"[it, s, primary, day, in, new, york, and, fron...","[primary, day, new, york, front, runners, hill...","[primary, day, new, york, front, runners, hill..."
...,...,...,...,...,...,...,...
6330,State Department says it can't find emails fro...,The State Department told the Republican Natio...,4076,the state department told the republican natio...,"[the, state, department, told, the, republican...","[state, department, told, republican, national...","[state, department, tell, republican, national..."
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,14323,the p in pbs should stand for plutocratic or p...,"[the, p, in, pbs, should, stand, for, plutocra...","[p, pbs, stand, plutocratic, pentagon, posted,...","[p, pbs, stand, plutocratic, pentagon, post, o..."
6332,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,11974,anti trump protesters are tools of the oligarc...,"[anti, trump, protesters, are, tools, of, the,...","[anti, trump, protesters, tools, oligarchy, re...","[anti, trump, protesters, tool, oligarchy, ref..."
6333,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",6991,addis ababa ethiopia president obama convened ...,"[addis, ababa, ethiopia, president, obama, con...","[addis, ababa, ethiopia, president, obama, con...","[addis, ababa, ethiopia, president, obama, con..."


In [31]:
#Creating a corpus of text feature to encode further into vectorized form
corpus= []
for row in dataset["Lemmatized_Text"]:
    msg = ' '.join(row)
    corpus.append(msg)

print("The First 10 lines in corpus :",*corpus[:10], sep = "\n")

The First 10 lines in corpus :
daniel greenfield shillman journalism fellow freedom center new york writer focus radical islam final stretch election hillary rodham clinton go war fbi word unprecedented throw around often election ought retire still unprecedented nominee major political party go war fbi exactly hillary people do coma patients wake watch hour cnn hospital bed would assume fbi director jam comey hillary opponent election fbi attack everyone obama cnn hillary people circulate letter attack comey currently media hit piece lambast target trump surprise clintons ally start run attack ads fbi fbi leadership warn entire leave wing establishment form lynch mob continue go hillary fbi credibility attack media democrats preemptively head result investigation clinton foundation hillary clinton covert struggle fbi agents obama doj people go explosively public new york time compare comey j edgar hoover bizarre headline jam comey role recall hoover fbi fairly practically admit front 

In [32]:
corpus

['daniel greenfield shillman journalism fellow freedom center new york writer focus radical islam final stretch election hillary rodham clinton go war fbi word unprecedented throw around often election ought retire still unprecedented nominee major political party go war fbi exactly hillary people do coma patients wake watch hour cnn hospital bed would assume fbi director jam comey hillary opponent election fbi attack everyone obama cnn hillary people circulate letter attack comey currently media hit piece lambast target trump surprise clintons ally start run attack ads fbi fbi leadership warn entire leave wing establishment form lynch mob continue go hillary fbi credibility attack media democrats preemptively head result investigation clinton foundation hillary clinton covert struggle fbi agents obama doj people go explosively public new york time compare comey j edgar hoover bizarre headline jam comey role recall hoover fbi fairly practically admit front spout nonsense boston globe p

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Changing text data in to numbers.
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(corpus).toarray()
#Let's have a look at our feature
X.dtype

dtype('float64')

In [34]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [35]:
from sklearn.preprocessing import LabelEncoder
#Label encode the Target and use it as y
label_encoder = LabelEncoder()
dataset["text"] = label_encoder.fit_transform(dataset["text"])

In [36]:
dataset

,title,text,text_len,Clean_Text,Tokenize_Text,Nostopword_Text,Lemmatized_Text
0,You Can Smell Hillary’s Fear,1514,7518,daniel greenfield a shillman journalism fellow...,"[daniel, greenfield, a, shillman, journalism, ...","[daniel, greenfield, shillman, journalism, fel...","[daniel, greenfield, shillman, journalism, fel..."
1,Watch The Exact Moment Paul Ryan Committed Pol...,2185,2646,google pinterest digg linkedin reddit stumbleu...,"[google, pinterest, digg, linkedin, reddit, st...","[google, pinterest, digg, linkedin, reddit, st...","[google, pinterest, digg, linkedin, reddit, st..."
2,Kerry to go to Paris in gesture of sympathy,5165,2543,u s secretary of state john f kerry said monda...,"[u, s, secretary, of, state, john, f, kerry, s...","[u, secretary, state, john, f, kerry, said, mo...","[u, secretary, state, john, f, kerry, say, mon..."
3,Bernie supporters on Twitter erupt in anger ag...,5991,2660,kaydee king kaydeeking november the lesson fro...,"[kaydee, king, kaydeeking, november, the, less...","[kaydee, king, kaydeeking, november, lesson, t...","[kaydee, king, kaydeeking, november, lesson, t..."
4,The Battle of New York: Why This Primary Matters,2733,1840,it s primary day in new york and front runners...,"[it, s, primary, day, in, new, york, and, fron...","[primary, day, new, york, front, runners, hill...","[primary, day, new, york, front, runners, hill..."
...,...,...,...,...,...,...,...
6330,State Department says it can't find emails fro...,4591,4076,the state department told the republican natio...,"[the, state, department, told, the, republican...","[state, department, told, republican, national...","[state, department, tell, republican, national..."
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,4861,14323,the p in pbs should stand for plutocratic or p...,"[the, p, in, pbs, should, stand, for, plutocra...","[p, pbs, stand, plutocratic, pentagon, posted,...","[p, pbs, stand, plutocratic, pentagon, post, o..."
6332,Anti-Trump Protesters Are Tools of the Oligarc...,217,11974,anti trump protesters are tools of the oligarc...,"[anti, trump, protesters, are, tools, of, the,...","[anti, trump, protesters, tools, oligarchy, re...","[anti, trump, protesters, tool, oligarchy, ref..."
6333,"In Ethiopia, Obama seeks progress on peace, se...",723,6991,addis ababa ethiopia president obama convened ...,"[addis, ababa, ethiopia, president, obama, con...","[addis, ababa, ethiopia, president, obama, con...","[addis, ababa, ethiopia, president, obama, con..."


In [37]:
from sklearn.model_selection import train_test_split

#Setting values for labels and feature as y and X(we already did X in vectorizing...)
y = dataset["text"]
# Splitting the testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

#Testing on the following classifiers
classifiers = [KNeighborsClassifier()]
for cls in classifiers:
    cls.fit(X_train, y_train)

# Dictionary of pipelines and model types for ease of reference
pipe_dict = {0:"KNeighbours"}

In [39]:
# Mengambil sampel dari data
X_sample, _, y_sample, _ = train_test_split(X, y, train_size=0.1, random_state=42)  # Menggunakan 10% dari data

In [40]:
from sklearn.model_selection import cross_val_score

# Cossvalidation
for i, model in enumerate(classifiers):
    cv_score = cross_val_score(model, X_train,y_train,scoring="accuracy", cv=10)
    print("%s: %f " % (pipe_dict[i], cv_score.mean()))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


KNeighbours: 0.037690 


In [41]:
# Testing on the following classifiers
classifiers = [MultinomialNB(),
               RandomForestClassifier(n_estimators=10),  # Mengurangi jumlah trees
               KNeighborsClassifier(n_neighbors=3),  # Mengurangi jumlah neighbors
               SVC()]

for cls in classifiers:
    cls.fit(X_sample, y_sample)  # Menggunakan data sampel

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Definisikan classifiers
classifiers = [MultinomialNB(),
               RandomForestClassifier(n_estimators=10),
               KNeighborsClassifier(n_neighbors=3),
               SVC()]

# Dictionary of pipelines and model types for ease of reference
pipe_dict = {0: "NaiveBayes", 1: "RandomForest", 2: "KNeighbours", 3: "SVC"}

# Mengambil sampel dari data
X_sample, _, y_sample, _ = train_test_split(X, y, train_size=0.1, random_state=42)  # Menggunakan 10% dari data

# Cross-validation
for i, model in enumerate(classifiers):
    # Menggunakan StratifiedKFold untuk menjaga proporsi kelas
    skf = StratifiedKFold(n_splits=5)  # Ubah n_splits menjadi 5
    cv_score = cross_val_score(model, X_sample, y_sample, scoring="accuracy", cv=skf)
    print("%s: %f " % (pipe_dict[i], cv_score.mean()))

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


NaiveBayes: 0.033208 


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


RandomForest: 0.042695 


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


KNeighbours: 0.033208 


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
